# MOC stream function analysis script

This script performs several analysis steps which are, within bounds, configurable to the users needs. The steps performed are listed here and documented below, where they are actually run. The Configuration for all analysis steps is located in the cell below, for convenience.

The tasks performed by this script are:
1. Create a plot for each timestep supplied in the netCDF-file both for the global MOC and all regions specified. Then creates a video of all these plots for each region.
2. Create a plot of the difference between two timesteps for each timestep, then create a video of those plots. For global MOC only.
3. Calculate and plot the average, min, max and standard deviation of the MOC stream function for each bin. Global MOC only.
4. Compares the values of the MOC stream function for each timestep within an interval _i_ to the averaged value for that interval and plots the differences. Both for the global and regional MOC.

## Configuration

*outputStyle* - Determines how the MOC streamfunction plots will be presented. Possible styles are _pdf_, _video_ and _both_. The default is _video_.

*subPlotsPerRow* - Determines the layout of the regional subplots (pdf files only). The default is 3.

*regionPlotPadding* - Determines how many of the bins outside of the actual latitude range of each region is being displayed in each regional plot. The default is 3.

*dataFilePath* - The path to the datafile used. The datafile must contain all the variables of the mocStreamfunction output stream.

*outputFileName* - The name of the output pdf file (pdf files only).

*outputDir* - The name of the general output directory. This is where your projects will be saved.

*diffDir* - The name of the folder the difference calculation results will be stored. The default is 'differences'.

*projectName* - The name of the current project. This determines the subfolder in _outputDir_ that all files generated will be stored.

*framesPerSecond* - Global configuration of the video-framerate used to encode the videos. The default is 10. Values can be any floating point number.

*differenceOutputFileName* - The name of the PDF file the difference plots will be saved.

*timeAverage* - Determines if, and, if yes, how many, timesteps are averaged for each plot.
The resulting number of plots will be `timestepsInRecord - timeAverage`. The default is 1 (no time averaging).

*stepping* - Determines, which plots will be created. `stepping = n` means, that every _n_ th timestep a plot will be created, averaging over timeAverage timesteps.

*colorScaleName* - Determines the colorscale used for all contour plots. The default is plasma. A list of available color scale names can be found at the respective code location.

In [ ]:
subPlotsPerRow = 3
regionPlotPadding = 10
outputStyle = 'both'
dataFilePath = '/Users/nilsfeige/mocStreamfunction.0000-01-01_30to60.nc'
outputFileName = 'mocDaily.pdf'
outputDir = 'outputs'
diffDir = 'differences'
projectName = 'signedEdge30to60'
framesPerSecond = 5
differenceOutputFileName = 'mocDifference.pdf'
timeAverage = 1
stepping = 1
# Accent, Accent_r, Blues, Blues_r, BrBG, BrBG_r, BuGn, BuGn_r, BuPu, BuPu_r, CMRmap, 
# CMRmap_r, Dark2, Dark2_r, GnBu, GnBu_r, Greens, Greens_r, Greys, Greys_r, OrRd, OrRd_r, 
# Oranges, Oranges_r, PRGn, PRGn_r, Paired, Paired_r, Pastel1, Pastel1_r, Pastel2, Pastel2_r,
# PiYG, PiYG_r, PuBu, PuBuGn, PuBuGn_r, PuBu_r, PuOr, PuOr_r, PuRd, PuRd_r, Purples, 
# Purples_r, RdBu, RdBu_r, RdGy, RdGy_r, RdPu, RdPu_r, RdYlBu, RdYlBu_r, RdYlGn, RdYlGn_r, 
# Reds, Reds_r, Set1, Set1_r, Set2, Set2_r, Set3, Set3_r, Spectral, Spectral_r, Wistia, 
# Wistia_r, YlGn, YlGnBu, YlGnBu_r, YlGn_r, YlOrBr, YlOrBr_r, YlOrRd, YlOrRd_r, afmhot, 
# afmhot_r, autumn, autumn_r, binary, binary_r, bone, bone_r, brg, brg_r, bwr, bwr_r, cool, 
# cool_r, coolwarm, coolwarm_r, copper, copper_r, cubehelix, cubehelix_r, flag, flag_r, 
# gist_earth, gist_earth_r, gist_gray, gist_gray_r, gist_heat, gist_heat_r, gist_ncar, 
# gist_ncar_r, gist_rainbow, gist_rainbow_r, gist_stern, gist_stern_r, gist_yarg, gist_yarg_r, 
# gnuplot, gnuplot2, gnuplot2_r, gnuplot_r, gray, gray_r, hot, hot_r, hsv, hsv_r, inferno, 
# inferno_r, jet, jet_r, magma, magma_r, nipy_spectral, nipy_spectral_r, ocean, ocean_r, pink, 
# pink_r, plasma, plasma_r, prism, prism_r, rainbow, rainbow_r, seismic, seismic_r, spectral, 
# spectral_r, spring, spring_r, summer, summer_r, terrain, terrain_r, viridis, viridis_r, 
# winter or winter_r
colorScaleName = 'spectral_r'

In [ ]:
%matplotlib inline

import sys, math
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
from array import *
from math import ceil
set_matplotlib_formats('png', 'pdf')
from colormap import *
from matplotlib.backends.backend_pdf import PdfPages
from datetime import *
import os
from subprocess import call
from statistics import mean, stdev

In [ ]:
ct = datetime.now()

## Data loading and preprocessing

Here the required data for all calculations is loaded from the netCDF file. Additionally, the currently selected region group and the (bin) boundaries of all regions are calculated.

In [ ]:
ds = nc.Dataset(dataFilePath)
mocStreamvalLatAndDepth = ds['mocStreamvalLatAndDepth'][:]
binBoundaries = ds['binBoundaryMocStreamfunction'][:]*180/math.pi
xtime = nc.chartostring(ds['xtime'][:])

regionsInGroup = ds['regionsInGroup'][:]
nRegionsInGroup = ds['nRegionsInGroup'][:]
regionNames = nc.chartostring(ds['regionNames'][:])
regionGroupNames = nc.chartostring(ds['regionGroupNames'][:])
regionMocData = ds['mocStreamvalLatAndDepthRegion'][:]
regionBoundaries = ds['minMaxLatRegion'][:]*180/math.pi

additionalGroupName = getattr(ds,'config_AM_mocStreamfunction_additionalRegion')
nRegionGroups = ds.dimensions['nRegionGroups'].size

for i in range(len(regionBoundaries[:,0])):
    if (regionBoundaries[i, 0] > regionBoundaries[i, 1]):
        temp = regionBoundaries[i, 0]
        regionBoundaries[i, 0] = regionBoundaries[i, 1]
        regionBoundaries[i, 1] = temp
    

for i in range(len(regionNames)):
    regionNames[i] = regionNames[i].strip()
    
for i in range(len(regionGroupNames)):
    regionGroupNames[i] = regionGroupNames[i].strip()
    
regionNumber = -1

for i in range(len(regionGroupNames)):
    if (regionGroupNames[i].decode('utf8') == additionalGroupName):
        curRegionGroup = i

numRegionsInCurGroup = nRegionsInGroup[curRegionGroup]

for i in range(len(xtime)):
    xtime[i] = xtime[i].strip()
    
rBD = ds['refBottomDepth'][:]*-1

regionBinNumber = [[0 for x in range(numRegionsInCurGroup)] for y in range(2)] 

for i in range(len(binBoundaries)-1): # for every bin
    for j in range(numRegionsInCurGroup): # and every group
        if regionBoundaries[j][0] > binBoundaries[i]: # find min
            regionBinNumber[0][j] = max(0, i - 1 - regionPlotPadding)
        if regionBoundaries[j][1] > binBoundaries[i + 1]: # find max
            regionBinNumber[1][j] = min(len(binBoundaries) - 1, i + regionPlotPadding)
            
numTimesteps = len(xtime)
            
#print numTimeSte
#print(regionBoundaries.shape)
#print(regionBoundaries)
#print(regionBinNumber)
#print(xtime.shape)
#print(mocStreamvalLatAndDepth.shape)
#print(binBoundaries.shape)
#print(rBD.shape)
#print(regionBoundaries.shape)
#print(regionMocData.shape)
#print(additionalGroupName)
#print(regionsInGroup)
#print(nRegionsInGroup)
#print(regionNames)
#print(regionGroupNames)
#print(nRegionGroups)
#print(curRegionGroup)
#print(numRegionsInCurGroup)

## Folder creation

As preparation for all outputs, the folders are created here.

In [ ]:
if not os.path.isdir(outputDir):
    os.mkdir(outputDir)

if not os.path.isdir(outputDir + "/" + projectName):
    os.mkdir(outputDir + "/" + projectName)
    
if not os.path.isdir(outputDir + '/' + projectName + '/global'):
    os.mkdir(outputDir + '/' + projectName + '/global')
    
if (not os.path.isdir(outputDir + '/' + projectName + '/' + diffDir)):
    os.mkdir(outputDir + '/' + projectName + '/' + diffDir)
    
if not os.path.isdir(outputDir + '/' + projectName + '/avgVSsingle'):
    os.mkdir(outputDir + '/' + projectName + '/avgVSsingle')

## Average vs SinglePlot comparison

For a given list of intervals, the average of the MOC stream function over that interval is calculated. Then, every timestep within that interval is compared to the average and the results, currently only min and max deviations, are stored. This is done for every timestep and every region. Calculations therefore may take a long time.

In [ ]:
times = [datetime.strptime(timestring.decode('utf-8'), '0000-%m-%d_%H:%M:%S') \
         for timestring in xtime]

intervals = {'day' : timedelta(1), 
             'week' : timedelta(7), 
             #'month' : timedelta(30), 
             #'year' : timedelta(365) 
            }

curIntervalNumber = 0
globalMax = []
globalMin = []
regionalMax = []
regionalMin = []

timestepGlobalMin = []
timestepGlobalMax = []
timestepRegionalMax = []
timestepRegionalMin = []
    
for interval in intervals:
    globalMax.append(0)
    globalMin.append(0)
    regionalMax.append([0 for x in range(numRegionsInCurGroup)])
    regionalMin.append([0 for x in range(numRegionsInCurGroup)])
    curInterval = intervals[interval]
    numTimestepsPerMainInterval = curInterval.total_seconds() // \
        (times[1] - times[0]).total_seconds()
    print('Starting calculation for interval:' + str(curInterval))
    
    i = 0
    j = 0
    lastPercent = 0
    
    timestepGlobalMin.append([])
    timestepGlobalMax.append([])
    timestepRegionalMax.append([])
    timestepRegionalMin.append([])
    timestepNumber = 0
    while (i < numTimesteps):

        mainAverageGlobal = mocStreamvalLatAndDepth[i, :, :]
        while (i + j < numTimesteps and times[i + j] - times[i] < curInterval):
            mainAverageGlobal = mainAverageGlobal + mocStreamvalLatAndDepth[i + j,:,:]
            j += 1
        actualAvgInterval = times[j - 1] - times[0]
        mainAverageGlobal /= j
        
        if (100 * i // numTimesteps > lastPercent):
            print(str(100 * i // numTimesteps) + '% of ' + str(curInterval) + ' completed')
            lastPercent = 100 * i // numTimesteps

        timestepGlobalMin[curIntervalNumber].append(0)
        timestepGlobalMax[curIntervalNumber].append(0)
        timestepRegionalMin[curIntervalNumber].append([0 for x in range(numRegionsInCurGroup)])
        timestepRegionalMax[curIntervalNumber].append([0 for x in range(numRegionsInCurGroup)])
        
        k = i
        while (k < numTimesteps and times[k] - times[i] < curInterval):
            plotValues = mainAverageGlobal[:,:] - mocStreamvalLatAndDepth[k, :, :]
            
            curMax = max([max(x) for x in plotValues])
            curMin = min([min(x) for x in plotValues])  
            
            globalMax[curIntervalNumber] = max(globalMax[curIntervalNumber], curMax)
            globalMin[curIntervalNumber] = min(globalMin[curIntervalNumber], curMin)
            
            timestepGlobalMin[curIntervalNumber][timestepNumber] = \
                min(timestepGlobalMin[curIntervalNumber][timestepNumber], curMin)
            timestepGlobalMax[curIntervalNumber][timestepNumber] = \
                max(timestepGlobalMax[curIntervalNumber][timestepNumber], curMax)
            for l in range(numRegionsInCurGroup):
                binSlice = slice(regionBinNumber[0][l], regionBinNumber[1][l])
                
                m = 1
                mainAverageLocal = regionMocData[i, l, :, binSlice]
                while (i + m < numTimesteps and times[i + m] - times[i] < curInterval):
                    mainAverageLocal = mainAverageLocal + regionMocData[i + m, l, :, binSlice]
                    m += 1
                mainAverageLocal /= m
                
                for n in range(m):
                    plotValues = mainAverageLocal[:,:] - regionMocData[i + n, l,:, binSlice]
                    
                    curMax = max([max(x) for x in plotValues])
                    curMin = min([min(x) for x in plotValues])
                    
                    timestepRegionalMax[curIntervalNumber][timestepNumber][l] = \
                        max(timestepRegionalMax[curIntervalNumber][timestepNumber][l], curMax)
                    timestepRegionalMin[curIntervalNumber][timestepNumber][l] = \
                        min(timestepRegionalMin[curIntervalNumber][timestepNumber][l], curMin)
                    
                    regionalMax[curIntervalNumber][l] = \
                        max(regionalMax[curIntervalNumber][l], curMax)
                    regionalMin[curIntervalNumber][l] = \
                        min(regionalMin[curIntervalNumber][l], curMin)
            k += 1
        i += j
        timestepNumber += 1
    curIntervalNumber += 1
    
#print(globalMax)
#print(regionalMax)

#print(timestepGlobalMax)
#print(timestepRegionalMax)

In [ ]:
plt.figure(1)
plt.clf()
plt.plot(globalMax, color='blue')
plt.plot(regionalMax, color='green')
plt.savefig(outputDir + '/' + projectName + '/avgVSsingle/globalAndRegionalMaxDev.png'\
           , format = 'png')

for i in range(len(intervals)):
    plt.clf()
    plt.plot(timestepGlobalMax[i][:][:], color='green')
    plt.plot(timestepRegionalMax[i][:][:], color='blue')
    plt.plot(timestepGlobalMin[i][:][:], color='red')
    plt.plot(timestepRegionalMin[i][:][:], color='yellow')
    plt.savefig(outputDir + '/' + projectName + '/avgVSsingle/minMaxGlobalAndRegion'\
               + str(i) + '.png', format = 'png')
    #plt.gca().set_ylim([-10,10])

## MOC plot creation

Here, plots for the global MOC as well as every region are created. It can be configured to output either a list of png files, from which a video is created at the end, or to generate a pdf file with one page for every timestep. In the pdf, every region has a smaller plot below the global MOC plot. The width of the subplots can be configured using the _subPlotsPerRow_ variable.

In [ ]:
isBoth = outputStyle == 'both'

isPDF = outputStyle == 'pdf' or isBoth
isVideo = outputStyle == 'video' or isBoth

if isPDF:
    pp = PdfPages(outputDir + '/' + projectName + '/' + outputFileName)
    numSubplotRows = math.ceil(numRegionsInCurGroup / subPlotsPerRow) * 2 + 4

videoParams = 18, 12
pdfParams = 18, 2 * numSubplotRows

def plotMOC(mode):
    _isVideo = False
    _isPDF = False
    if mode == 'pdf':
        plt.rcParams['figure.figsize'] = pdfParams
        _isPDF = True
    if mode == 'video':
        plt.rcParams['figure.figsize'] = videoParams
        _isVideo = True
        
    lastPercent = 0
    for i in range(0, len(xtime) - timeAverage + 1, stepping):
        mainAverage = mocStreamvalLatAndDepth[i, :, :]
        for j in range(1, timeAverage):
            mainAverage = mainAverage + mocStreamvalLatAndDepth[i + j,:,:]
        mainAverage /= timeAverage
        plt.figure(1, tight_layout=True)
    
        if (100 * i // numTimesteps > lastPercent):
            lastPercent = 100 * i // numTimesteps
            print(str(lastPercent) + '% completed')
        
        if (_isPDF):
            plt.subplot2grid((numSubplotRows, subPlotsPerRow), (0, 0), \
                             colspan=subPlotsPerRow, rowspan=4)
        try:
            contourSet = plt.contour(binBoundaries, rBD, mainAverage, linewidths = 0.5, \
                                    colors="black")
        except ValueError:
            continue
        plt.contourf(binBoundaries, rBD, mainAverage)
        plt.set_cmap(colorScaleName)
        cb = plt.colorbar()
        plt.clabel(contourSet, colors="black")
        plt.title('Global MOC by Latitude and Depth [time: ' + xtime[i].decode('utf8') + ']')
        plt.xlabel('latitude [deg]')
        plt.ylabel('Depth [m]')
        subplotCounter = 0
        
        if _isVideo:
            plt.savefig(outputDir + '/' + projectName + '/global/plot' + str(i).zfill(5) \
                        + '.png', format='png')
    
        for j in range(numRegionsInCurGroup):
            regionSlice = slice(regionBinNumber[0][j], regionBinNumber[1][j])
            regionName = regionNames[regionsInGroup[curRegionGroup][j] - 1].decode('utf8')
            regionBinBoundaries = binBoundaries[regionSlice]
            if (not os.path.isdir(outputDir + '/' + projectName + '/' + regionName)):
                os.mkdir(outputDir + '/' + projectName + '/' + regionName)
            
            if _isVideo:
                plt.figure(1)
                plt.clf()
                
            mainAverage = regionMocData[i, j, :, regionSlice]
            for k in range(1, timeAverage):
                mainAverage = mainAverage + regionMocData[i + k, j, :, regionSlice]
            mainAverage /= timeAverage
            curRow = math.floor(j / subPlotsPerRow) * 2 + 4
            curColumn = math.floor(j % subPlotsPerRow)
            
            if _isPDF:
                plt.subplot2grid((numSubplotRows, subPlotsPerRow), (curRow, curColumn), \
                                 rowspan=2)
            
            cs = plt.contour(regionBinBoundaries, rBD, \
                             mainAverage, \
                             linewidths = 0.5, colors="black", \
                             extent=contourSet.extent, extend='both')
            plt.contourf(regionBinBoundaries, rBD, \
                         mainAverage, \
                         extent=contourSet.extent, extend='both')
            plt.set_cmap(colorScaleName)
            cb = plt.colorbar()
            plt.clabel(cs, colors="black")
            plt.title('Regional MOC [region: "' + regionName \
                      + '"]' + '[time: ' + xtime[i].decode('utf8') + ']')
            plt.xlabel('latitude [deg]')
            plt.ylabel('Depth [m]')
            if _isVideo:
                plt.savefig(outputDir + '/' + projectName + '/' + regionName + \
                            '/plot' + str(i).zfill(5) + '.png' \
                            , format='png')
        if _isPDF:
            pp.savefig()
        plt.clf()
    if _isPDF:
        pp.close()
        
if (isBoth):
    plotMOC('video')
    plotMOC('pdf')
else:
    plotMOC(outputStyle)

In [ ]:
print("Video mode enabled:", isVideo)
if isVideo:
    call(['/Users/nilsfeige/pyana/anaconda/bin/ffmpeg', '-f', 'image2', '-r', \
          str(framesPerSecond), '-i', outputDir + '/' + projectName + \
          '/global/plot%05d.png', '-y', '-codec', 'mpeg4', '-b:v', '40000k', \
          outputDir + '/' + projectName + '/globalMovie.mp4'])
    
    print('Finished global video.')

    for j in range(numRegionsInCurGroup):
        regionName = regionNames[regionsInGroup[curRegionGroup][j] - 1].decode('utf8')
        call(['/Users/nilsfeige/pyana/anaconda/bin/ffmpeg', '-f', 'image2', '-r', \
              str(framesPerSecond), '-i', outputDir + '/' + projectName + '/' + regionName \
              + '/plot%05d.png', '-y', '-codec', 'mpeg4', '-b:v', '40000k', outputDir + \
              '/' + projectName + '/' + regionName + 'Movie.mp4'])
        print('Finished video for region ' + regionName + '.')

## Statistical analysis

Here, the min and max values for each bin are calculated for each timestep. The results are plotted as a series of images and then converted to a video. Also, the difference in the MOC between each timestep and its successor is calculated, plotted as contours and output as a series of images. The images then are converted into a video file.

Then, a plot showing the average and the standard deviation for each bin over all timesteps is written to a file. In a last step, the minimum and maximum value of each timestep is plotted against time.

In [ ]:
plt.rcParams['figure.figsize'] = 22, 12
    
numBins = len(binBoundaries)
maxBinValue = []
minBinValue = []

plt.figure(1)
lastPercent = 0
for i in range(len(xtime)):
    maxBinValue.append([])
    minBinValue.append([])
    if (100 * i // numTimesteps > lastPercent):
        lastPercent = 100 * i // numTimesteps
        print(str(lastPercent) + '% completed')
    for j in range(numBins):
        tempData = mocStreamvalLatAndDepth[i,:,j]
        maxBinValue[i].append(max(tempData))
        minBinValue[i].append(min(tempData))
    plt.clf()
    plt.plot(binBoundaries, maxBinValue[i], color='black')
    plt.plot(binBoundaries, minBinValue[i], color='blue')
    plt.savefig(outputDir + '/' + projectName + '/' + diffDir + '/binVals' + \
                str(i).zfill(5) + '.png', format='png')

plt.clf()
print('minmax calculation done')

plt.figure(2)
lastPercent = 0
for i in range(len(xtime) - 1):
    if (100 * i // numTimesteps > lastPercent):
        lastPercent = 100 * i // numTimesteps
        print(str(lastPercent) + '% completed')
    
    values = mocStreamvalLatAndDepth[i, :, :] - mocStreamvalLatAndDepth[i + 1, :, :]
    
    plt.clf()
    cs = plt.contour(binBoundaries, rBD, values, \
                     linewidths=0.5, colors="black"\
                     ,extend='both')
    plt.contourf(binBoundaries, rBD, \
                 values, \
                 extent=cs.extent, extend='both')
    plt.set_cmap(colorScaleName)
    b = plt.colorbar()
    plt.clabel(cs, colors="black")
    plt.title('Successive step MOC differences ' + str(i).zfill(4))
    plt.xlabel('latitude [deg]')
    plt.ylabel('Depth [m]')
    plt.savefig(outputDir + '/' + projectName + '/' + diffDir + '/diffVals' + \
                str(i).zfill(5) + '.png', format='png')
plt.clf()

maxValueOverTime = [max(x) for x in maxBinValue]
minValueOverTime = [min(x) for x in minBinValue]

plt.figure(1)
plt.clf()
plt.plot(maxValueOverTime, color='blue')
plt.plot(minValueOverTime, color='red')
plt.savefig(outputDir + '/' + projectName + '/' + diffDir + '/minMaxBinOTime.png', \
            format='png')

avgMax = []
avgMin = []
stddMax = []
stddMin = []

for i in range(numBins):
    avgMax.append(mean([x[i] for x in maxBinValue]))
    avgMin.append(mean([x[i] for x in minBinValue]))
    stddMax.append(stdev([x[i] for x in maxBinValue]))
    stddMin.append(stdev([x[i] for x in minBinValue]))
    
plt.figure(1)
plt.clf()
plt.plot(avgMax, color='blue')
plt.plot(avgMin, color='red')
plt.plot(stddMax, color='turquoise')
plt.plot(stddMin, color='purple')
plt.savefig(outputDir + '/' + projectName + '/' + diffDir + '/avgAndDevOverBin.png', \
            format='png')
plt.clf()

print('succession calculation done')

In [ ]:
call(['/Users/nilsfeige/pyana/anaconda/bin/ffmpeg', '-f', 'image2', '-r', \
      str(framesPerSecond), '-i', outputDir + '/' + projectName + '/' + diffDir +\
      '/diffVals%05d.png', '-y', '-codec', 'mpeg4', \
      '-b:v', '40000k', outputDir + '/' + projectName + '/diffMovie.mp4'])

print("Finished video for forward differences.")

call(['/Users/nilsfeige/pyana/anaconda/bin/ffmpeg', '-f', 'image2', '-r', \
      str(framesPerSecond), '-i', outputDir + '/' + projectName + '/' + diffDir \
      + '/binVals%05d.png', '-y', '-codec', 'mpeg4', \
      '-b:v', '40000k', outputDir + '/' + projectName + '/minMaxBinMovie.mp4'])

print("Finished video for minMax per Bin.")

In [ ]:
ct2 = datetime.now()
print(ct2 -ct)